# Lab 3 — dimensionality reduction

**Group:** W

**Names:**

* Olivier Cloux
* Thibault Urien
* Saskia Reiss

In [1]:
# Given imports
import json
import matplotlib.pyplot as plt
import numpy as np

# Import panda that is easier to use than dicts
import pandas as pd

from operator import itemgetter

%matplotlib inline
plt.style.use("ggplot")

In [ ]:
# Let's see whats inside those files to use them later.
!hdfs dfs -cat /ix/ml-20m/genome-tags.txt | tail -n 1
!hdfs dfs -cat /ix/ml-20m/movies.txt | tail -n 1
!hdfs dfs -cat /ix/ml-20m/genome-scores.txt | tail -n 1
!hdfs dfs -cat /ix/ml-20m/ratings.txt | tail -n 1

In [2]:
# Given code for the dict of all tag names
data = sc.textFile("/ix/ml-20m/genome-tags.txt").map(json.loads)
#tag2name = dict(data.map(itemgetter("tagId", "tag")).collect())

# Data Frame of all tag names
tag2nameDF = pd.DataFrame(data.collect())

### Introduction to the Data
To begin this lab, we choose to answer to the first questions asked in the handout, which are:
* How many different movies are there? How many tags, tag scores, movie ratings?
* How many movies have at least one tag?
* How many different movie genres are there?
* What are the shortest and longest movie titles? Tag names?

Note that we use panda, as we had a few problems with dictionnaries that we could not resolve. The Data Frames given by Panda being quite easy to use too.

In [3]:
# Data Frame of all movies
data1 = sc.textFile("/ix/ml-20m/movies.txt").map(json.loads)
moviesDF = pd.DataFrame(data1.map(itemgetter("genres", "movieId", "title")).collect())

In [4]:
# Data Frame of all tag scores
data2 = sc.textFile("/ix/ml-20m/genome-scores.txt").map(json.loads)
tagScoresDF = pd.DataFrame(data2.map(itemgetter("relevance", "tagId", "movieId")).collect())

In [ ]:
# Data Frame of all movie ratings
data3 = sc.textFile("/ix/ml-20m/ratings.txt").map(json.loads)
data3.map(itemgetter("movieId", "userId", "timestamp", "rating")).collect()
#ratingsDF = pd.DataFrame(data3.map(itemgetter("movieId", "userId", "timestamp", "rating")).collect())